## Ising model

In this notebook, we calculate low lying energy eigenstates of an Ising chain using DMRG, and measure some observables in it.

In [1]:
using Plots, TensorNetworks
plotlyjs()

┌ Info: Precompiling TensorNetworks [bf05fd6d-b814-440f-8e78-cc5435f5625f]
└ @ Base loading.jl:1342


WebIO._IJuliaInit()

Plots.PlotlyJSBackend()

In [2]:
chain_length = 100
physical_dim = 2 
Dmax = 50
prec = 1e-12
truncationTEBD = TruncationArgs(Dmax, prec, true)
truncationDMRG = TruncationArgs(Dmax, prec, true)
J = 1.0
h = .1
g = 0; #Small field to break degeneracy

Hmpo = IsingMPO(chain_length,J,h,g)
Hgates = isingHamGates(chain_length,J,h,g);

n_states = 4

4

In [3]:
initialmps = randomOpenPVMPS(chain_length,2,Dmax;truncation = truncationTEBD)
state = TensorNetworks.imaginaryTEBD(initialmps,Hgates,10,.1)

-0.1--0.2--0.3--0.4--0.5--0.6--0.7--0.8--0.9--1.0-

MPS: OpenMPS{ComplexF64}
Sites: OrthogonalLinkSite{ComplexF64}
Length: 100
Truncation: TruncationArgs(50, 1.0e-12, true)

In [4]:
shifter = SubspaceExpand(10.0)
initialmps2 = canonicalize(LCROpenMPS(state))
set_center!(initialmps2,1)
initialmps2.truncation = truncationDMRG
states, energies = eigenstates(Hmpo, initialmps2, n_states, precision = prec, shifter = shifter);

E, var = -99.25516289865064, 6.771188054699451e-8
E, var, ΔE/E = -99.25516291571873, -2.1827872842550278e-11, -1.71961726351038e-10
E, var, ΔE/E = -99.25516291571887, -9.094947017729282e-12, -1.4317496740465735e-15
  8.399398 seconds (24.56 M allocations: 1.506 GiB, 4.56% gc time, 49.29% compilation time)
E, var = -99.25516289865064, 6.771188054699451e-8
E, var, ΔE/E = -99.2551629157195, -5.093170329928398e-11, -1.7196945779927652e-10
E, var, ΔE/E = -99.2551629157191, -5.4569682106375694e-12, 4.008899087330396e-15
  1.044675 seconds (2.42 M allocations: 291.723 MiB, 6.06% gc time, 26.03% compilation time)
E, var = -99.25516289865064, 6.771188054699451e-8
E, var, ΔE/E = -97.45311255901314, 8.423765393672511e-6, 0.018491460070567418
E, var, ΔE/E = -97.45373321455634, 5.260480975266546e-6, -6.368720034922783e-6
E, var, ΔE/E = -97.45401426578206, 1.608656020835042e-6, -2.883936878787472e-6
E, var, ΔE/E = -97.454310703751, 7.173530320869759e-7, -3.041814844319883e-6
E, var, ΔE/E = -97.45443

In [5]:
magGate = Gate(sz)
mag = [real.(expectation_values(state, magGate)) for state in states];

domainwallGate = Gate(reshape(kron(sz,sz), (2,2,2,2)))
domainwall = [real.(1 .- expectation_values(state, domainwallGate))/2 for state in states];

es = [real.([expectation_value(state, Hgates[k],k) for k in 1:chain_length-1]) for state in states];

entropy = [TensorNetworks.entanglement_entropy(state) for state in states];

In [6]:
normalization = sqrt(sum(mag[1])^2 + sum(mag[2])^2)
zero_mag_state = (sum(mag[1])/normalization, states[1]) + (sum(mag[2])/normalization,states[2])
magzero = real.(expectation_values(zero_mag_state, magGate));

In [7]:
pmag = plot(hcat(mag...),xlabel="position", ylabel = "<Z>")
add_trace!(pmag, plot(magzero))

LoadError: UndefVarError: add_trace! not defined

In [ ]:
plot([magzero], xlabel="position", ylabel = "<Z>")

In [ ]:
plot(domainwall, xlabel="position", ylabel = "Domain wall density")

In [ ]:
plot(es, xlabel="position", ylabel = "energy")

In [ ]:
plot(real.(entropy), xlabel="position", ylabel = "entanglement entropy")